In [164]:
import numpy as np
import pandas as pd
import plotly.offline as pyo
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly import tools

In [126]:
match = pd.read_csv("C:\\Users\\ACER\\Desktop\\DSMP1\\Datasets\\datasets-session-20\\matches.csv")
delivery = pd.read_csv("C:\\Users\\ACER\\Desktop\\DSMP1\\Datasets\\datasets-session-20\\deliveries.csv")

ipl =  delivery.merge(match, left_on="match_id", right_on="id")
ipl.head()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,...,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,...,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,...,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,...,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,...,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,...,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN


## Scatter Plot

In [127]:
# Problem :- Draw a scatter plot between Batsman Avg(X axis) and Batsman Strike Rate(Y axis) of top 50 Batsman of all time.


# Extracting Top 50 Batsman
top50 = ipl.groupby('batsman')['batsman_runs'].sum().sort_values(ascending=False).head(50).index.to_list()
new_ipl = ipl[ipl['batsman'].isin(top50)]

In [128]:
# Calculating Strike Rate

runs = new_ipl.groupby('batsman')['batsman_runs'].sum()
balls = new_ipl.groupby('batsman')['batsman_runs'].count()

sr = (runs/balls)*100

sr = sr.reset_index()
sr.head()

,batsman,batsman_runs
0,AB de Villiers,145.129059
1,AC Gilchrist,133.054662
2,AJ Finch,126.299213
3,AM Rahane,117.486549
4,AT Rayudu,123.014257


In [129]:
# Calculating Average

out = ipl[ipl['player_dismissed'].isin(top50)]

nouts = out['player_dismissed'].value_counts()

avg = runs/nouts

avg = avg.reset_index()
avg.rename(columns={'index' :'batsman', 0: 'avg'}, inplace=True)

avg = avg.merge(sr, on='batsman')
avg.head()

,batsman,avg,batsman_runs
0,AB de Villiers,38.307692,145.129059
1,AC Gilchrist,27.223684,133.054662
2,AJ Finch,27.186441,126.299213
3,AM Rahane,33.593407,117.486549
4,AT Rayudu,27.146067,123.014257


In [130]:
# Plot Scatter Plot Here

trace = go.Scatter(
            x = avg['avg'], 
            y=avg['batsman_runs'],
            mode='markers',
            hovertext=avg['batsman'],
            marker=dict(
                size=16,
                color='red',
                opacity=0.8,
                line=dict(width=0.5, color='black')
    )
)

data = [trace]

layout = go.Layout(
            title=' Batsman Average VS Strike Rate',
            xaxis={
                'title': 'Batsman Average'
            },
            yaxis={
                'title': 'Batsman Strike Rate'
            },
)

fig = go.Figure(data, layout).update_layout(
        autosize=False,  
        width=1700,      
        height=900,      
)

fig.show()

## Line Chart

In [131]:
# Year by Year Batsman Performance

single = ipl[ipl['batsman']=='V Kohli']
performance = single.groupby('season')['batsman_runs'].sum().reset_index()
performance

single1 = ipl[ipl['batsman']=='MS Dhoni']
performance1 = single1.groupby('season')['batsman_runs'].sum().reset_index()
performance1

,season,batsman_runs
0,2008,414
1,2009,332
2,2010,287
3,2011,392
4,2012,357
5,2013,461
6,2014,371
7,2015,372
8,2016,284
9,2017,290


In [132]:
# Plot the Line Chart

trace = go.Scatter(
    x=performance['season'],
    y=performance['batsman_runs'],
    mode='lines + markers',
    name='V Kohli',
    marker=dict(
                size=16,
                color='green',
                opacity=0.8,
                line=dict(width=0.5, color='black')
    )
)

trace1 = go.Scatter(
    x=performance1['season'],
    y=performance1['batsman_runs'],
    mode='lines + markers',
    name='MS Dhoni',
    marker=dict(
                size=16,
                color='red',
                opacity=0.8,
                line=dict(width=0.5, color='black')
    )
)

data=[trace, trace1]

layout = go.Layout(
            title=' Year by Year Performance',
            xaxis={
                'title': 'Season'
            },
            yaxis={
                'title': 'Total Runs'
            },
)

fig = go.Figure(data, layout).update_layout(
        autosize=False,  
        width=1700,      
        height=900,      
)

fig.show()

In [133]:
# Multiple Line Charts

def batsman_comp(*name):
    data=[]
    for i in name:
        single = ipl[ipl['batsman']== i ]
        performance = single.groupby('season')['batsman_runs'].sum().reset_index()
        
        trace = go.Scatter(
            x=performance['season'],
            y=performance['batsman_runs'],
            mode='lines + markers',
            name=i,
            marker=dict(
                size=16,
                opacity=0.8,
                line=dict(width=0.5, color='black')
            )
        )
        
        data.append(trace)
        
    layout = go.Layout(
            title=' Year by Year Performance',
            xaxis={
                'title': 'Season'
            },
            yaxis={
                'title': 'Total Runs'
            },
    )
    
    fig = go.Figure(data, layout).update_layout(
        autosize=False,  
        width=1700,      
        height=900,      
    )

    fig.show()

In [134]:
batsman_comp('V Kohli', 'RG Sharma', 'DA Warner', 'MS Dhoni')

## Bar Plot

In [135]:
top10 = ipl.groupby('batsman')['batsman_runs'].sum().sort_values(ascending=False).head(10).index.to_list()
top10_df = ipl[ipl['batsman'].isin(top10)]

In [136]:
top10_score = top10_df.groupby('batsman')['batsman_runs'].sum().reset_index()
top10_score

,batsman,batsman_runs
0,AB de Villiers,3486
1,CH Gayle,3651
2,DA Warner,4014
3,G Gambhir,4132
4,MS Dhoni,3560
5,RG Sharma,4207
6,RV Uthappa,3778
7,S Dhawan,3561
8,SK Raina,4548
9,V Kohli,4423


In [137]:
# Plot Bar Chart


trace = go.Bar(
    x=top10_score['batsman'],
    y=top10_score['batsman_runs'],    
)

data=[trace]

layout = go.Layout(
            title= 'Top 10 IPL Batsman',
            xaxis={
                'title': 'Batsman'
            },
            yaxis={
                'title': 'Total Runs'
            },
    )
    
fig = go.Figure(data, layout).update_layout(
        autosize=False,  
        width=1700,      
        height=900,      
    )

fig.show()

#### There are 2 types of Bar Graphs

1. Nested Bar Graph
2. Stacked Bar Graph
3. Overlayed Bar Graph

In [138]:
iw = top10_df.groupby(['batsman','inning'])['batsman_runs'].sum().reset_index()
mask = iw['inning'] == 1
mask2 = iw['inning'] == 2

one = iw[mask]
two = iw[mask2]

one.rename(columns={'batsman_runs':'1st Innings'}, inplace=True)
two.rename(columns={'batsman_runs':'2nd Innings'}, inplace=True)

final = one.merge(two,on='batsman')[['batsman','1st Innings','2nd Innings']]

final

,batsman,1st Innings,2nd Innings
0,AB de Villiers,2128,1345
1,CH Gayle,2003,1623
2,DA Warner,2118,1896
3,G Gambhir,1699,2433
4,MS Dhoni,2232,1328
5,RG Sharma,2344,1863
6,RV Uthappa,1516,2262
7,S Dhawan,2262,1299
8,SK Raina,2647,1893
9,V Kohli,2391,2027


In [139]:
# Stacked

trace1 = go.Bar(
    x=final['batsman'],
    y=final['1st Innings'],
    name='1st Innings',
    marker={'color':'#00a65a'}
)

trace2 = go.Bar(
    x=final['batsman'],
    y=final['2nd Innings'],
    name='2nd Innings',
    marker={'color':'#a6a65a'}
)

data=[trace1, trace2]

layout = go.Layout(
            title= 'Innings Wise Score',
            xaxis={
                'title': 'Batsman'
            },
            yaxis={
                'title': 'Runs'
            },
            barmode='stack'
    )
    
fig = go.Figure(data, layout).update_layout(
        autosize=False,  
        width=1700,      
        height=900,      
    )

fig.show()

In [140]:
# Overlay

trace1 = go.Bar(
    x=final['batsman'],
    y=final['1st Innings'],
    name='1st Innings',
    marker={'color':'#00a65a'}
)

trace2 = go.Bar(
    x=final['batsman'],
    y=final['2nd Innings'],
    name='2nd Innings',
    marker={'color':'#a6a65a'}
)

data=[trace1, trace2]

layout = go.Layout(
            title= 'Innings Wise Score',
            xaxis={
                'title': 'Batsman'
            },
            yaxis={
                'title': 'Runs'
            },
            barmode='overlay'
    )
    
fig = go.Figure(data, layout).update_layout(
        autosize=False,  
        width=1700,      
        height=900,      
    )

fig.show()

In [141]:
# Nested

trace1 = go.Bar(
    x=final['batsman'],
    y=final['1st Innings'],
    name='1st Innings',
    marker={'color':'#00a65a'}
)

trace2 = go.Bar(
    x=final['batsman'],
    y=final['2nd Innings'],
    name='2nd Innings',
    marker={'color':'#a6a65a'}
)

data=[trace1, trace2]

layout = go.Layout(
            title= 'Innings Wise Score',
            xaxis={
                'title': 'Batsman'
            },
            yaxis={
                'title': 'Runs'
            },
    )
    
fig = go.Figure(data, layout).update_layout(
        autosize=False,  
        width=1700,      
        height=900,      
    )

fig.show()

## Bubble Plot

In [142]:
new_ipl = new_ipl[new_ipl['batsman_runs'] == 6]

six = new_ipl.groupby('batsman')['batsman_runs'].count().reset_index()

x = avg.merge(six, on='batsman')

trace = go.Scatter(
    x=x['avg'],
    y=x['batsman_runs_x'],
    mode='markers',
    # name=x['batsman'],
    marker={
        'size':x['batsman_runs_y']
    }
)

data=[trace]

layout = go.Layout(
            title= 'Bubble Chart',
            xaxis={
                'title': 'Average'
            },
            yaxis={
                'title': 'Strike Rate'
            },
    )
    
fig = go.Figure(data, layout).update_layout(
        autosize=False,  
        width=1700,      
        height=900,      
    )

fig.show()

## Boxplot

In [143]:
match_agg = delivery.groupby(['match_id'])['total_runs'].sum().reset_index()
season_wise = match_agg.merge(match, left_on='match_id', right_on='id')[['match_id', 'total_runs', 'season']]
season_wise

,match_id,total_runs,season
0,1,379,2017
1,2,371,2017
2,3,367,2017
3,4,327,2017
4,5,299,2017
...,...,...,...
631,632,277,2016
632,633,317,2016
633,634,302,2016
634,635,325,2016


In [144]:
trace=go.Box(
    x=season_wise['total_runs'],
    name='All Seasons',
    marker={
        'color':'#00a65a'
    }
)

data=[trace]

layout = go.Layout(
            title= 'Total Score Analysis',
            xaxis={
                'title': 'Total Score'
            },
            
    )
    
fig = go.Figure(data, layout).update_layout(
        autosize=False,  
        width=1700,      
        height=900,      
    )

fig.show()

In [145]:
trace1=go.Box(
    x=season_wise[season_wise['season'] == 2017]['total_runs'],
    name='2017',
    marker={
        'color':'#00a65a'
    }
)

trace2=go.Box(
    x=season_wise[season_wise['season'] == 2008]['total_runs'],
    name='2008',
    
)

data=[trace1, trace2]

layout = go.Layout(
            title= 'Total Score Analysis',
            xaxis={
                'title': 'Total Score'
            },
            
    )
    
fig = go.Figure(data, layout).update_layout(
        autosize=False,  
        width=1700,      
        height=900,      
    )

fig.show()

## Distplot

In [152]:
hist_data = [avg['avg'], avg['batsman_runs']]

group_labels = ['Average', 'Strike Rate']

fig = ff.create_distplot(hist_data, group_labels, bin_size=[3,5]).update_layout(
        autosize=False,  
        width=1700,      
        height=900,      
    )

fig.show()

## Histogram

In [154]:
x = delivery.groupby('batsman')['batsman_runs'].count() > 150
x = x[x].index.tolist()

new = delivery[delivery['batsman'].isin(x)]

runs = new.groupby('batsman')['batsman_runs'].sum()
balls = new.groupby('batsman')['batsman_runs'].count()

sr = (runs/balls)*100

sr = sr.reset_index()
sr

,batsman,batsman_runs
0,A Ashish Reddy,142.857143
1,A Mishra,89.005236
2,A Symonds,124.711908
3,AA Jhunjhunwala,99.541284
4,AB Agarkar,111.875000
...,...,...
157,Y Nagar,105.166052
158,Y Venugopal Rao,113.872832
159,YK Pathan,140.751445
160,YV Takawale,104.918033


In [160]:
# Plot Histogram

trace=go.Histogram(
    x=sr['batsman_runs'],
    xbins={'size': 2, 'start':50, 'end':100}
)

data=[trace]

layout = go.Layout(
            title= 'Strike Rate Analysis',
            xaxis={
                'title': 'Strike Rates'
            },
    )
    
fig = go.Figure(data, layout).update_layout(
        autosize=False,  
        width=1700,      
        height=900,      
    )

fig.show()

## HeatMaps

In [161]:
six = delivery[delivery['batsman_runs'] == 6]
six = six.groupby(['batting_team','over'])['batsman_runs'].count().reset_index()

six

,batting_team,over,batsman_runs
0,Chennai Super Kings,1,5
1,Chennai Super Kings,2,17
2,Chennai Super Kings,3,37
3,Chennai Super Kings,4,34
4,Chennai Super Kings,5,41
...,...,...,...
271,Sunrisers Hyderabad,16,22
272,Sunrisers Hyderabad,17,18
273,Sunrisers Hyderabad,18,37
274,Sunrisers Hyderabad,19,42


In [162]:
# Plot Heatmap

trace = go.Heatmap(
    x=six['batting_team'],
    y=six['over'],
    z=six['batsman_runs']
)

data=[trace]

layout = go.Layout(
            title= 'Strike Rate Analysis',
            
    )
    
fig = go.Figure(data, layout).update_layout(
        autosize=False,  
        width=1700,      
        height=900,      
    )

fig.show()

In [163]:
dots = delivery[delivery['batsman_runs'] == 0]
dots = dots.groupby(['batting_team','over'])['batsman_runs'].count().reset_index()

dots

,batting_team,over,batsman_runs
0,Chennai Super Kings,1,524
1,Chennai Super Kings,2,462
2,Chennai Super Kings,3,432
3,Chennai Super Kings,4,421
4,Chennai Super Kings,5,376
...,...,...,...
275,Sunrisers Hyderabad,16,144
276,Sunrisers Hyderabad,17,157
277,Sunrisers Hyderabad,18,128
278,Sunrisers Hyderabad,19,130


In [167]:
trace1 = go.Heatmap(
    x=six['batting_team'],
    y=six['over'],
    z=six['batsman_runs'].values.tolist(),
    text=six['batsman_runs'].values.tolist(),
    texttemplate="%{text}",
    textfont={"size": 12, "color": "white"},
    colorscale="Viridis"
)

trace2 = go.Heatmap(
    x=dots['batting_team'],
    y=dots['over'],
    z=dots['batsman_runs'].values.tolist(),
    text=dots['batsman_runs'].values.tolist(), 
    texttemplate="%{text}",
    textfont={"size": 12, "color": "white"},
    colorscale="Plasma"
)

fig = tools.make_subplots(rows=1, cols=2, subplot_titles = ["6's","0's"], shared_yaxes=True).update_layout(
        autosize=False,  
        width=1700,      
        height=900,      
    )

fig.append_trace(trace1,1,1)
fig.append_trace(trace2,1,2)


fig.show()